# Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 500

# Import CSV's into pandas dataframes

In [2]:
df_submissions = pd.read_csv("reddit_submissions_2018-04-25_2034.csv",
skipinitialspace=True,
header = 0,
sep = ',',
index_col=0,
encoding='latin1').fillna('')
print(df_submissions.shape)
print(df_submissions.columns)

df_comments = pd.read_csv("reddit_comments_2018-04-25_2034.csv",
skipinitialspace=True,
header = 0,
sep = ',',
index_col=0,
encoding='latin1').fillna('')
print(df_comments.shape)
print(df_comments.columns)

(1363, 24)
Index(['approved_by', 'archived', 'author', 'author_flair_text',
       'comment_limit', 'created', 'distinguished', 'downs', 'gilded', 'id',
       'is_video', 'likes', 'name', 'num_comments', 'num_crossposts',
       'over_18', 'pinned', 'quarantine', 'score', 'selftext', 'send_replies',
       'subreddit_id', 'title', 'ups'],
      dtype='object')
(6671, 18)
Index(['_submission', 'approved_by', 'archived', 'author', 'author_flair_text',
       'body', 'comment_id', 'created', 'distinguished', 'downs', 'gilded',
       'link_id', 'name', 'parent_id', 'score', 'send_replies', 'subreddit_id',
       'ups'],
      dtype='object')


# Renaming Columns

In [3]:
df_comments = df_comments.rename(columns={'_submission':'id',
                                         'body':'selftext'})

# Merge Dataframes 

In [4]:
df_all = df_submissions.append(df_comments)
df_all['comment'] = np.where(df_all['comment_id'].isnull(),0,1)
print(df_all.shape)
df_all.columns

(8034, 28)


Index(['approved_by', 'archived', 'author', 'author_flair_text', 'comment_id',
       'comment_limit', 'created', 'distinguished', 'downs', 'gilded', 'id',
       'is_video', 'likes', 'link_id', 'name', 'num_comments',
       'num_crossposts', 'over_18', 'parent_id', 'pinned', 'quarantine',
       'score', 'selftext', 'send_replies', 'subreddit_id', 'title', 'ups',
       'comment'],
      dtype='object')

In [5]:
print('***print submission***')
print('Head {}'.format(df_all['comment'][0:5]))
print('***print comments***')
print('Bottom {}'.format(df_all['comment'][-5:-1]))

***print submission***
Head 0    0
1    0
2    0
3    0
4    0
Name: comment, dtype: int64
***print comments***
Bottom 0    1
0    1
0    1
0    1
Name: comment, dtype: int64


### Count # of users

In [6]:
print(df_all['id'].nunique())
print(df_all['author'].nunique())
print(df_all['selftext'].nunique())

1014
1024
8029


# Search in the text by

- submission_id
- comment_id

In [13]:
pattern = 'security'
df_all['selftext'] = df_all['selftext'].str.lower()
df_pattern = df_all[df_all['selftext'].str.contains(pattern.lower())]
print(df_pattern['selftext'][0:5])

528    my wife is pregnant and will soon be going on mat leave. last year she used income averaging so she could spend some more time with our son and will fibish paying it off just before maf leave starts.\r\r\n\r\r\ndoes anyone have any experience going on mat leave following income averaging? when u submitted your best weeks pay for ei did you provide your pay before income averaging was taken off or did you provide your pay without income averaging?\r\r\n\r\r\nfor example say she made 2000 gros...
590    hi friends, like most of you here i am experiencing a lot of pay issues. iâ?ve decided to go back to when we were all moved over to phoenix (may 2016), and calculate my pay and deductions (i.e. gross vs net), and things are going sideways very quickly. \r\r\n\r\r\ncit, cpp, and ei that were deducted from my gross pay are not the same as the calculations i am making based on the rates posted on the cra website for the 2016 tax year. for example, on one of my pays it appears as thoug

In [8]:
def find_subm_comment(df):
    pattern_comment_id = []
    pattern_subm_id = []
    for index, row in df.iterrows():
        if row['comment']==0:
            pattern_comment_id.append(df_pattern['comment_id'])
        if row['comment']==1:
            pattern_subm_id.append(df_pattern['comment_id'])
    return pattern_comment_id, pattern_subm_id

In [9]:
pattern_comment_id, pattern_subm_id = find_subm_comment(df_pattern)

print('Number of Submissions or comments with the "{}": {} '.format(pattern,len(df_pattern)))
print('Number of Submissions with the "{}": {} '.format(pattern,len(pattern_subm_id)))
print('Number of comments with the "{}": {} '.format(pattern,len(pattern_comment_id)))

Number of Submissions or comments with the "security": 270 
Number of Submissions with the "security": 193 
Number of comments with the "security": 77 


# Extract Key Words of submission or comments

In [10]:
important_names = pd.read_csv("Important_Names.csv",
    skipinitialspace=True,
    header = 0,
    sep = ',',
    encoding='latin1').fillna('')

print(important_names.columns)
important_names = list(important_names['Very important'])

while '' in important_names:
    important_names.remove('')

program_names = pd.read_csv("program_names.csv",
    skipinitialspace=True,
    header = 0,
    sep = ',',
    encoding='latin1').fillna('')

program_names = list(program_names['names'])

Index(['Very important', 'Somewhat important'], dtype='object')


### Programs Names

In [11]:
list_occurences = []
for word in program_names:
    pattern = word
    df_pattern = df_all[df_all['selftext'].str.contains(pattern.lower())]
    pattern_comment_id, pattern_subm_id = find_subm_comment(df_pattern)
    list_occurences.append([len(pattern_comment_id), len(pattern_subm_id)])

/Users/Ale/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


In [12]:
dict_program_name = dict(zip(program_names,list_occurences))
for key, values in dict_program_name.items():
    print('{} --> {}'.format(key, values))

passport --> [0, 11]
other government department programs --> [0, 0]
old age security --> [0, 2]
canada pension plan --> [0, 0]
the canada pension plan disability --> [0, 0]
canada disability savings program --> [0, 0]
national child benefit --> [0, 0]
labour relations --> [2, 20]
canada student loans and grants --> [0, 0]
canada education savings program --> [0, 0]
early learning and child care --> [0, 0]
indigenous early learning and child care --> [0, 0]
canadian poverty reduction --> [0, 0]
social innovation and social finance --> [0, 0]
government of canada telephone general enquiries services --> [0, 0]
government of canada internet presence --> [0, 0]
in-person points of service --> [0, 0]
employment insurance --> [0, 4]
labour market agreements for persons with disabilities --> [0, 0]
opportunities fund for persons with disabilities --> [0, 0]
youth employment strategy --> [0, 0]
enabling fund for official language minority communities --> [0, 0]
aboriginal skills and employmen

### Important Names

In [30]:
print(type(important_names[0]))
print(important_names)

<class 'str'>
['Josée Duplessis', 'Emilie Gauduchon', 'Mathieu Filion', 'Matthew Mitschke', 'Matt Pascuzzo', 'Carlene Variyan', 'Jude Welch', 'Annabelle Archambault', 'Jane Almeida', 'Adam Vaughan', 'Rodger Cuzner', 'Stéphane Lauzon', 'Peter Schiefke', 'Jean-Yves Duclos', 'Duclos', 'Patty Hajdu', 'Hajdu', 'Kirsty Duncan', 'Duncan', 'Kent Hehr', 'Hehr', 'Christopher Simard', 'Olivier Bouffard', 'Amelie Maisonneuve', 'Joshua Bueckert', 'Carla Qualtrough', 'Qualtrough']


In [31]:
list_occurences = []
for word in important_names:
    pattern = word
    df_pattern = df_all[df_all['selftext'].str.contains(pattern.lower())]
    pattern_comment_id, pattern_subm_id = find_subm_comment(df_pattern)
    list_occurences.append([len(pattern_comment_id), len(pattern_subm_id)])

In [32]:
dict_important_names = dict(zip(important_names,list_occurences))
for key, values in dict_important_names.items():
    print('{} --> {}'.format(key, values))

Josée Duplessis --> [0, 0]
Emilie Gauduchon --> [0, 0]
Mathieu Filion --> [0, 0]
Matthew Mitschke --> [0, 0]
Matt Pascuzzo --> [0, 0]
Carlene Variyan --> [0, 0]
Jude Welch --> [0, 0]
Annabelle Archambault --> [0, 0]
Jane Almeida --> [0, 0]
Adam Vaughan --> [0, 0]
Rodger Cuzner --> [0, 0]
Stéphane Lauzon --> [0, 0]
Peter Schiefke --> [0, 0]
Jean-Yves Duclos --> [0, 0]
Duclos --> [0, 0]
Patty Hajdu --> [0, 0]
Hajdu --> [0, 0]
Kirsty Duncan --> [0, 0]
Duncan --> [0, 0]
Kent Hehr --> [0, 0]
Hehr --> [0, 0]
Christopher Simard --> [0, 0]
Olivier Bouffard --> [0, 0]
Amelie Maisonneuve --> [0, 0]
Joshua Bueckert --> [0, 0]
Carla Qualtrough --> [0, 6]
Qualtrough --> [0, 20]


# Keywords ...

In [97]:
from gensim.summarization import keywords

C:\Users\alex_\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Clustering - Mean Shift